<a href="https://colab.research.google.com/github/sushrut88/Machine-Learning-Lab/blob/main/practical3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>